# Operators

## Preliminaries

Let's start by importing Wick&d and defining a Slater determinant reference

In [1]:
import wicked as w
from IPython.display import display, Math, Latex

def latex(expr):
    """Function to render any object that has a member latex() function"""
    display(Math(expr.latex()))
    
w.reset_space()
w.add_space("o", "fermion", "occupied", ['i','j','k','l','m'])
w.add_space("v", "fermion", "unoccupied", ['a','b','c','d','e','f'])    

## Canonical representation of operators in wick&d

Wick&d defines a **canonical form** for operators. A canonical operator satisfies the following conditions:
- creation operators are to the left of annihilation operators
- all operators are normal ordered with respect to the reference state
- the numerical tensor that multiplies a product of second quantized operators is antisymmetric with respect to permutation of upper (annihilation) and lower (creation) indices

In this case, you only need to indicate which type of second quantized operators and which orbital space they act on to fully specify a general operator.

For example, the one-body operator
\begin{equation}
\sum_{ij}^\mathbb{O} h_{i}^{j} \{ \hat{a}^{\dagger}_{i} \hat{a}_{j} \}
\equiv \sum_{ij}^\mathbb{O} h_{i}^{j} \{ \hat{a}^{i}_{j} \}
\end{equation}
can be created using the `op` function and passing the operator label (`h`) and a list of second quantized operators that appear in it (`['o+ o']`)

In [2]:
Hoo = w.op('h',['o+ o'])
Hoo

+ h { o+ o }

If we specify the operators in a noncanonical order, Wick&d will resort the operators and **ignore any sign change**:

In [3]:
Hoo = w.op('h',['o o+'])
Hoo

+ h { o+ o }

The following two-electron operator
\begin{equation}
\frac{1}{4} \sum_{ij}^\mathbb{O} \sum_{ab}^\mathbb{V} v_{ij}^{ab} \{  \hat{a}^{\dagger}_{i} \hat{a}^{\dagger}_{j} \hat{a}_{b} \hat{a}_{a} \}
\end{equation}
can be requested with

In [4]:
Voovv = w.op('v',['o+ o+ v v'])
Voovv

+ 1/4 v { o+ o+ v v }

Note that we did not have to specify the numerical prefactor (1/4) since wick&d can determine it automatically by counting the number of equivalent operators

If an operator involves multiple orbital spaces, like the following operator
\begin{equation}
\sum_{ij}^\mathbb{O} f_{i}^{j} \{ \hat{a}^{i}_{j} \}
+ \sum_{ab}^\mathbb{V} f_{a}^{b} \{ \hat{a}^{a}_{b} \}
\end{equation}
we can just pass a list with all the components

In [5]:
Fd = w.op('f',['o+ o','v+ v'])
Fd

+ f { v+ v }
+ f { o+ o }

## Defining operators via the utlility function `gen_op` 

When we want to specify an operator that contains all the possible combination of second quantized operators we can use the `gen_op` function.
A general one-body operator with all components can be generated via:

In [6]:
w.utils.gen_op('F',1,'ov','ov')

+ F { v+ v }
+ F { v+ o }
+ F { o+ v }
+ F { o+ o }

While a two-body operator can be obtained via

In [7]:
w.utils.gen_op('V',2,'ov','ov')

+ 1/4 V { v+ v+ v v }
+ 1/2 V { v+ v+ v o }
+ 1/4 V { v+ v+ o o }
+ 1/2 V { o+ v+ v v }
+ V { o+ v+ v o }
+ 1/2 V { o+ v+ o o }
+ 1/4 V { o+ o+ v v }
+ 1/2 V { o+ o+ v o }
+ 1/4 V { o+ o+ o o }

We can also exclude diagonal components via the optional argument `diagonal=False`

In [8]:
w.utils.gen_op('F',1,'ov','ov',diagonal=False)

+ F { v+ o }
+ F { o+ v }

## Operator expressions and manipulation

Wick&d provides several functions to handle operator expressions (linear combinations of products of operators).
For example, operators can be multiplied using the noncommutative multiplication operation (`@`)

In [9]:
A = w.op('A',['o+ v'])
B = w.op('B',['v+ o'])
C = w.op('C',['o+ o'])
AB = A @ B
AB

+ A { o+ v } B { v+ o }

We can also compute the commutator of two operators using the `commutator` function
\begin{equation}
[\hat{A},\hat{B}]
\end{equation}

In [10]:
AB_comm = w.commutator(A,B)
AB_comm

+ A { o+ v } B { v+ o }
- B { v+ o } A { o+ v }

The function `commutator` accepts an arbitrary number of arguments, and can be used to evaluate nested commutators of the form
\begin{equation}
[\ldots[[\hat{A},\hat{B}],\hat{C}],\ldots]
\end{equation}
For example the commutator $[[\hat{A},\hat{B}],\hat{C}]$ can be evaluated as

In [11]:
AB_comm = w.commutator(A,B,C)
AB_comm

+ A { o+ v } B { v+ o } C { o+ o }
- B { v+ o } A { o+ v } C { o+ o }
- C { o+ o } A { o+ v } B { v+ o }
+ C { o+ o } B { v+ o } A { o+ v }

We can also compute truncated expansion of the Baker–Campbell–Hausdorff formula
\begin{equation}
\exp(-\hat{B})\hat{A}\exp(\hat{B}) = \hat{A} + \frac{1}{1!} [\hat{A},\hat{B}]+
+ \frac{1}{2!} [[\hat{A},\hat{B}],\hat{B}]
+ \frac{1}{3!} [[[\hat{A},\hat{B}],\hat{B}],\hat{B}] + \cdots
\end{equation}
For example, the series truncated to two commutators is given by

In [12]:
AB_bch = w.bch_series(A,B,2)
AB_bch

+ A { o+ v }
+ A { o+ v } B { v+ o }
+1/2 A { o+ v } B { v+ o } B { v+ o }
- B { v+ o } A { o+ v }
- B { v+ o } A { o+ v } B { v+ o }
+1/2 B { v+ o } B { v+ o } A { o+ v }

When operators commute, one can simplify operator expressions by bringing all terms in a canonical order

In [13]:
C = w.op('C',['o+ o'])
D = w.op('D',['v+ v'])
CD_bch = w.bch_series(C,D,4)
CD_bch

+ C { o+ o }
+ C { o+ o } D { v+ v }
+1/2 C { o+ o } D { v+ v } D { v+ v }
+1/6 C { o+ o } D { v+ v } D { v+ v } D { v+ v }
+1/24 C { o+ o } D { v+ v } D { v+ v } D { v+ v } D { v+ v }
- D { v+ v } C { o+ o }
- D { v+ v } C { o+ o } D { v+ v }
-1/2 D { v+ v } C { o+ o } D { v+ v } D { v+ v }
-1/6 D { v+ v } C { o+ o } D { v+ v } D { v+ v } D { v+ v }
+1/2 D { v+ v } D { v+ v } C { o+ o }
+1/2 D { v+ v } D { v+ v } C { o+ o } D { v+ v }
+1/4 D { v+ v } D { v+ v } C { o+ o } D { v+ v } D { v+ v }
-1/6 D { v+ v } D { v+ v } D { v+ v } C { o+ o }
-1/6 D { v+ v } D { v+ v } D { v+ v } C { o+ o } D { v+ v }
+1/24 D { v+ v } D { v+ v } D { v+ v } D { v+ v } C { o+ o }

In this case the canonical ordering leads to cancellation of all product terms

In [14]:
CD_bch.canonicalize()
CD_bch

+ C { o+ o }